In [99]:
#librerías
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
sys.path.append(r"C:\Users\mfpen\Documents\Repositorios\geoloc2\preparacion_datos\src")
from preparacion_inter_puntos import prep

Lectura de la base de CURT correspondiente al municipio (Tipo shp)

In [120]:
#base de curt y manzanas municipales
CURT_mun=gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp")
manzanas_mun = gpd.read_file(r"D:\Secretaria\INFORMACION PARA INTELIGENCIA FISCAL\ATLACOMULCO\SHAPE\MANZANAS\024 Atlacomulco\2023 024 MZ.shp")
#reemplazar id cat por clave catastral
CURT_mun['CLAVECATASTRAL']= CURT_mun['id_cat']
manzanas_mun['CLAVECATASTRAL']= manzanas_mun['cve_cat']

In [ ]:
#Proceso para base solo con manzanas
manzanas_mun = gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\Full manzanas\Full manzanas\Manzana_Temoaya.shp")
manzanas_mun['CLAVECATASTRAL']= manzanas_mun['cve_cat']
manzanas_mun

In [122]:
#revisar si hay vacíos en clave catastrar en CURT y Manzanas
manzanas_mun[manzanas_mun.CLAVECATASTRAL.isna()]
CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL


In [123]:
#Revisar cuantas CC se duplican
CURT_mun.CLAVECATASTRAL.value_counts()

0240801089000000    3
0240144233000000    2
0240321506000000    2
0240126306000000    2
0240132706000000    2
                   ..
0240332925000000    1
0240332906000000    1
0240332943000000    1
0240332911000000    1
0240122339000000    1
Name: CLAVECATASTRAL, Length: 1886, dtype: int64

In [124]:
#Disolución de las geometrías a partir de clavecatastral 
CURT_mun=CURT_mun.dissolve(by='CLAVECATASTRAL').reset_index()
CURT_mun.tail(2)
#Proceso solo para base con manzanas
# manzanas_mun=manzanas_mun.dissolve(by='CLAVECATASTRAL').reset_index()
# manzanas_mun


,CLAVECATASTRAL,geometry,cve_ent,cve_mun,cve_loc,id_cat,curt,notas
1884,0241530822000000,"POLYGON ((-99.84697 19.80856, -99.84700 19.808...",15,024,0000,0241530822000000,194830782609950493744,None
1885,0241530824000000,"POLYGON ((-99.84688 19.80852, -99.84698 19.808...",15,024,0000,0241530824000000,194830235009950491335,None


In [125]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs Manzanas: ",manzanas_mun.crs)

crs CURT:  EPSG:4326
crs Manzanas:  EPSG:32614


In [126]:
#Se le asigna el crs de CURT a la base de manzanas
manzanas_mun=manzanas_mun.to_crs(4326)
# Se hace un overlay para la unión de las bases 
CURT_mun=CURT_mun.overlay(manzanas_mun, how='union',keep_geom_type=False)
CURT_mun.tail(5)

,CLAVECATASTRAL_1,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,CLAVECATASTRAL_2,geometry
4307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814..."
4308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795..."
4309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792..."
4310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ..."
4311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,024,02,127,0240212700000000,0240212700000000,"POLYGON ((-99.85801 19.78996, -99.85697 19.789..."


In [127]:
CURT_mun['CLAVECATASTRAL_1']=CURT_mun.CLAVECATASTRAL_1.combine_first(CURT_mun.CLAVECATASTRAL_2)
CURT_mun

,CLAVECATASTRAL_1,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,CLAVECATASTRAL_2,geometry
0,0240100601000000,15,024,0000,0240100601000000,194804678009952240860,None,024,01,006,0240100600000000,0240100600000000,"POLYGON ((-99.87328 19.80097, -99.87348 19.801..."
1,0240100822000000,15,024,0000,0240100822000000,194805341109952221128,None,024,01,008,0240100800000000,0240100800000000,"POLYGON ((-99.87276 19.80146, -99.87278 19.801..."
2,0240100827000000,15,024,0000,0240100827000000,194805279409952217160,None,024,01,008,0240100800000000,0240100800000000,"POLYGON ((-99.87278 19.80143, -99.87276 19.801..."
3,0240100828000000,15,024,0000,0240100828000000,194806038509952206157,None,024,01,008,0240100800000000,0240100800000000,"POLYGON ((-99.87240 19.80161, -99.87245 19.801..."
4,0240101114000000,15,024,0000,0240101114000000,194814880409952222562,None,024,01,011,0240101100000000,0240101100000000,"POLYGON ((-99.87265 19.80403, -99.87306 19.804..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,0240323900000000,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814..."
4308,0240141000000000,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795..."
4309,0240134000000000,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792..."
4310,0240135200000000,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ..."


In [128]:
def combine_first_drop(gdf):
    for i in (gdf.columns):
        if str('_1') in i :
            n = i
        if str('_2') in i :
            k = i
            z = i[:i.rfind('_1')-1]
            gdf[z] = gdf[n]
            gdf.drop(columns=[n,k],inplace=True)
        else:
            pass

In [129]:
#Se utiliza esta función para renombrar columna con _1 y eliminar la otra
combine_first_drop(CURT_mun)
CURT_mun

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL
0,15,024,0000,0240100601000000,194804678009952240860,None,024,01,006,0240100600000000,"POLYGON ((-99.87328 19.80097, -99.87348 19.801...",0240100601000000
1,15,024,0000,0240100822000000,194805341109952221128,None,024,01,008,0240100800000000,"POLYGON ((-99.87276 19.80146, -99.87278 19.801...",0240100822000000
2,15,024,0000,0240100827000000,194805279409952217160,None,024,01,008,0240100800000000,"POLYGON ((-99.87278 19.80143, -99.87276 19.801...",0240100827000000
3,15,024,0000,0240100828000000,194806038509952206157,None,024,01,008,0240100800000000,"POLYGON ((-99.87240 19.80161, -99.87245 19.801...",0240100828000000
4,15,024,0000,0240101114000000,194814880409952222562,None,024,01,011,0240101100000000,"POLYGON ((-99.87265 19.80403, -99.87306 19.804...",0240101114000000
...,...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814...",0240323900000000
4308,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795...",0240141000000000
4309,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792...",0240134000000000
4310,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ...",0240135200000000


In [130]:
def separar_digitos(prueba):

    prueba.CLAVECATASTRAL = prueba.CLAVECATASTRAL.astype(str)
    for i in prueba.CLAVECATASTRAL:
        if str('000000') in i:
            cat = i
            cat=cat[:-6]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat
        elif str('00000000') in i :
            cat2=i
            cat2=cat2[:-8]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat2
        else:
            pass


In [131]:
#Se utiliza esta función para separar CC en CC manzana y CC predio
separar_digitos(CURT_mun)
CURT_mun

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL
0,15,024,0000,0240100601000000,194804678009952240860,None,024,01,006,0240100600000000,"POLYGON ((-99.87328 19.80097, -99.87348 19.801...",0240100601000000,0240100601
1,15,024,0000,0240100822000000,194805341109952221128,None,024,01,008,0240100800000000,"POLYGON ((-99.87276 19.80146, -99.87278 19.801...",0240100822000000,0240100822
2,15,024,0000,0240100827000000,194805279409952217160,None,024,01,008,0240100800000000,"POLYGON ((-99.87278 19.80143, -99.87276 19.801...",0240100827000000,0240100827
3,15,024,0000,0240100828000000,194806038509952206157,None,024,01,008,0240100800000000,"POLYGON ((-99.87240 19.80161, -99.87245 19.801...",0240100828000000,0240100828
4,15,024,0000,0240101114000000,194814880409952222562,None,024,01,011,0240101100000000,"POLYGON ((-99.87265 19.80403, -99.87306 19.804...",0240101114000000,0240101114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,"POLYGON ((-99.92435 19.81447, -99.92433 19.814...",0240323900000000,0240323900
4308,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,"POLYGON ((-99.86629 19.79529, -99.86647 19.795...",0240141000000000,0240141000
4309,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,"POLYGON ((-99.86166 19.79180, -99.86237 19.792...",0240134000000000,0240134000
4310,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,"MULTIPOLYGON (((-99.86557 19.78974, -99.86539 ...",0240135200000000,0240135200


In [132]:
CURT_mun = CURT_mun.set_geometry('geometry')
CURT_mun['geometry']=CURT_mun.centroid

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\2139259433.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  CURT_mun['geometry']=CURT_mun.centroid


In [133]:
CURT_mun['LATITUD']= CURT_mun['geometry'].y
CURT_mun['LONGITUD']= CURT_mun['geometry'].x
CURT_mun

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL,LATITUD,LONGITUD
0,15,024,0000,0240100601000000,194804678009952240860,None,024,01,006,0240100600000000,POINT (-99.87333 19.80121),0240100601000000,0240100601,19.801213,-99.873334
1,15,024,0000,0240100822000000,194805341109952221128,None,024,01,008,0240100800000000,POINT (-99.87279 19.80149),0240100822000000,0240100822,19.801491,-99.872794
2,15,024,0000,0240100827000000,194805279409952217160,None,024,01,008,0240100800000000,POINT (-99.87271 19.80146),0240100827000000,0240100827,19.801457,-99.872712
3,15,024,0000,0240100828000000,194806038509952206157,None,024,01,008,0240100800000000,POINT (-99.87239 19.80168),0240100828000000,0240100828,19.801677,-99.872393
4,15,024,0000,0240101114000000,194814880409952222562,None,024,01,011,0240101100000000,POINT (-99.87285 19.80407),0240101114000000,0240101114,19.804073,-99.872847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,NaN,NaN,NaN,024,03,239,0240323900000000,POINT (-99.92313 19.81402),0240323900000000,0240323900,19.814022,-99.923128
4308,NaN,NaN,NaN,NaN,NaN,NaN,024,01,410,0240141000000000,POINT (-99.86598 19.79625),0240141000000000,0240141000,19.796248,-99.865982
4309,NaN,NaN,NaN,NaN,NaN,NaN,024,01,340,0240134000000000,POINT (-99.86302 19.79344),0240134000000000,0240134000,19.793441,-99.863022
4310,NaN,NaN,NaN,NaN,NaN,NaN,024,01,352,0240135200000000,POINT (-99.86646 19.79088),0240135200000000,0240135200,19.790885,-99.866461


In [ ]:
#Correr solo si no se tiene curt
manzanas_mun = manzanas_mun.set_geometry('geometry')
manzanas_mun['geometry']=manzanas_mun.centroid

In [ ]:
#Correr solo si no se tiene curt
manzanas_mun['LATITUD']= manzanas_mun['geometry'].y
manzanas_mun['LONGITUD']= manzanas_mun['geometry'].x
manzanas_mun

In [134]:
#Se vuelven a revisar
CURT_mun[CURT_mun.geometry.isna()]
# CURT_mun[CURT_mun.CLAVECATASTRAL.isna()]

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL,LATITUD,LONGITUD


In [135]:
CURT_mun = CURT_mun.loc[~CURT_mun['manz'].astype(str).str.endswith('000')]
CURT_mun.shape

(4305, 15)

In [137]:
CURT_mun.to_file(r"D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_curtxmanzanas.shp")

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\2769763012.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  CURT_mun.to_file(r"D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_curtxmanzanas.shp")
Normalized/laundered field name: 'CLAVECATASTRAL' to 'CLAVECATAS'
Normalized/laundered field name: 'CLAVE_CATASTRAL' to 'CLAVE_CATA'


In [138]:
base_predial_OD = pd.read_excel(r"D:\Secretaria\cruces_bases\Atlacomulco\predial_diciembre_atlacomulco.xlsx")
base_predial_OD.tail(3)

,fol_rec,nombre,rfc,cve_catastral,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,gast_ejec,descuento,fol_fisc_uuid,facturado
23138,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,431987,DE 01-2021 AL 12-2021,2021,2022-12-30,574.0,212.0,0,0,212.0,NaN,NaN
23139,TM049725,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,455005,DE 01-2022 AL 12-2022,2022,2022-12-30,606.0,90.0,0,0,90.0,NaN,NaN
23140,TM049726,COLIN FLORES ELOISA*,XAXX010101000,0240119818000000,LEGISLATURA S/N,NaN,152.0,0.0,115520,DE 01-2022 AL 12-2022,2022,2022-12-30,208.0,35.0,0,0,35.0,NaN,NaN


In [139]:
base_predial_OD=base_predial_OD.reset_index(drop=True)
base_predial_OD

,fol_rec,nombre,rfc,cve_catastral,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,gast_ejec,descuento,fol_fisc_uuid,facturado
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747,DE 01-2022 AL 12-2022,2022,2022-01-03,196.0,0.0,0,0,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
1,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,02401271010F0014,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,0.0,148771,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,0.0,0,0,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.0,58.0,621149,DE 01-2022 AL 12-2022,2022,2022-01-03,864.0,0.0,0,0,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.0,251.0,867817,DE 01-2022 AL 12-2022,2022,2022-01-03,1356.0,0.0,0,0,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.0,79.0,295397,DE 01-2022 AL 12-2022,2022,2022-01-03,384.0,0.0,0,0,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,382309,DE 01-2019 AL 12-2019,2019,2022-12-30,504.0,252.0,0,0,252.0,NaN,NaN
23137,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,411399,DE 01-2020 AL 12-2020,2020,2022-12-30,544.0,272.0,0,0,272.0,NaN,NaN
23138,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,431987,DE 01-2021 AL 12-2021,2021,2022-12-30,574.0,212.0,0,0,212.0,NaN,NaN
23139,TM049725,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,455005,DE 01-2022 AL 12-2022,2022,2022-12-30,606.0,90.0,0,0,90.0,NaN,NaN


In [ ]:
base_predial_OD.drop(['Número Consecutivo'],axis=1,inplace=True)

In [141]:
#Se puede utilizar esta celda en caso de que la CC no este homologada a CLAVECATASTRAL
base_predial_OD.rename(columns={'cve_catastral': 'CLAVECATASTRAL'}, inplace=True)
base_predial_OD

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,gast_ejec,descuento,fol_fisc_uuid,facturado
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747,DE 01-2022 AL 12-2022,2022,2022-01-03,196.0,0.0,0,0,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
1,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,02401271010F0014,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,0.0,148771,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,0.0,0,0,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.0,58.0,621149,DE 01-2022 AL 12-2022,2022,2022-01-03,864.0,0.0,0,0,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.0,251.0,867817,DE 01-2022 AL 12-2022,2022,2022-01-03,1356.0,0.0,0,0,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.0,79.0,295397,DE 01-2022 AL 12-2022,2022,2022-01-03,384.0,0.0,0,0,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,382309,DE 01-2019 AL 12-2019,2019,2022-12-30,504.0,252.0,0,0,252.0,NaN,NaN
23137,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,411399,DE 01-2020 AL 12-2020,2020,2022-12-30,544.0,272.0,0,0,272.0,NaN,NaN
23138,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,431987,DE 01-2021 AL 12-2021,2021,2022-12-30,574.0,212.0,0,0,212.0,NaN,NaN
23139,TM049725,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,455005,DE 01-2022 AL 12-2022,2022,2022-12-30,606.0,90.0,0,0,90.0,NaN,NaN


In [153]:
#Secrea una base con los registros que SI tienen CC en predial
base_predial=base_predial_OD[base_predial_OD.CLAVECATASTRAL.notna()]
base_predial

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,gast_ejec,descuento,fol_fisc_uuid,facturado
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747,DE 01-2022 AL 12-2022,2022,2022-01-03,196.0,0.0,0,0,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
1,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,02401271010F0014,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,0.0,148771,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,0.0,0,0,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.0,58.0,621149,DE 01-2022 AL 12-2022,2022,2022-01-03,864.0,0.0,0,0,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.0,251.0,867817,DE 01-2022 AL 12-2022,2022,2022-01-03,1356.0,0.0,0,0,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.0,79.0,295397,DE 01-2022 AL 12-2022,2022,2022-01-03,384.0,0.0,0,0,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,382309,DE 01-2019 AL 12-2019,2019,2022-12-30,504.0,252.0,0,0,252.0,NaN,NaN
23137,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,411399,DE 01-2020 AL 12-2020,2020,2022-12-30,544.0,272.0,0,0,272.0,NaN,NaN
23138,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,431987,DE 01-2021 AL 12-2021,2021,2022-12-30,574.0,212.0,0,0,212.0,NaN,NaN
23139,TM049725,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,455005,DE 01-2022 AL 12-2022,2022,2022-12-30,606.0,90.0,0,0,90.0,NaN,NaN


In [187]:
#Se crean la CC a nivel manzana y a nivel predio de la base predial 
base_predial['CLAVE_MANZ']=base_predial.CLAVECATASTRAL.astype(str).str[0:8]+"00"
base_predial['CLAVE_PREDIO']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial['CLAVE_CATASTRAL']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
base_predial.tail(3)

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\3618090253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_predial['CLAVE_MANZ']=base_predial.CLAVECATASTRAL.astype(str).str[0:8]+"00"
C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\3618090253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_predial['CLAVE_PREDIO']=base_predial.CLAVECATASTRAL.astype(str).str[0:10]
C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\3618090253.py:4: SettingWithCopyWarning: 
A value is trying to be set on a c

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,...,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,EJERCICIO_FISCAL,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL
23131,TM049719,MONTOYA CRUZ MAURILIO,XAXX010101000,0240132737000000,"PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL","PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL",93.0,142.0,446012.0,DE 01-2022 AL 12-2022,...,594.0,87.0,NaN,NaN,NaN,NaN,2022,0240132700,0240132737,0240132737
23135,TM049723,DIAZ RAMIREZ JESUS*,XAXX010101000,0240119813000000,LEGISLATURA,NaN,130.0,186.0,744601.0,DE 01-2022 AL 12-2022,...,1086.0,120.0,NaN,120.0,NaN,NaN,2022,0240119800,0240119813,0240119813
23140,TM049726,COLIN FLORES ELOISA*,XAXX010101000,0240119818000000,LEGISLATURA S/N,NaN,152.0,NaN,115520.0,DE 01-2022 AL 12-2022,...,208.0,35.0,NaN,35.0,NaN,NaN,2022,0240119800,0240119818,0240119818


In [188]:
#quitar duplicados
base_predial.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
base_predial

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\4127517136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_predial.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)


,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,...,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,EJERCICIO_FISCAL,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747.0,DE 01-2022 AL 12-2022,...,196.0,NaN,NaN,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0241101600,0241101614,0241101614
1,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,02401271010F0014,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,...,219.0,NaN,NaN,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240127100,0240127101,0240127101
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.0,58.0,621149.0,DE 01-2022 AL 12-2022,...,864.0,NaN,NaN,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240210200,0240210222,0240210222
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.0,251.0,867817.0,DE 01-2022 AL 12-2022,...,1356.0,NaN,NaN,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240102200,0240102207,0240102207
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.0,79.0,295397.0,DE 01-2022 AL 12-2022,...,384.0,NaN,NaN,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240211000,0240211027,0240211027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23129,TM049715,JIMENEZ CHAVEZ MARIA DEL REFUGIO*,XAXX010101000,0240121208000000,MORELOS JOSE MARIA PTE. COL. LA GARITA,NaN,239.0,309.0,1328705.0,DE 01-2022 AL 12-2022,...,2646.0,293.0,NaN,900.0,NaN,NaN,2022,0240121200,0240121208,0240121208
23130,TM049716,JIMENEZ DE MORALES MA. DEL REFUGIO*,XAXX010101000,0240152401000000,3 SANTA CRUZ BOMBATEVI,NaN,1463.0,80.0,866723.0,DE 01-2022 AL 12-2022,...,1356.0,151.0,NaN,NaN,NaN,NaN,2022,0240152400,0240152401,0240152401
23131,TM049719,MONTOYA CRUZ MAURILIO,XAXX010101000,0240132737000000,"PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL","PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL",93.0,142.0,446012.0,DE 01-2022 AL 12-2022,...,594.0,87.0,NaN,NaN,NaN,NaN,2022,0240132700,0240132737,0240132737
23135,TM049723,DIAZ RAMIREZ JESUS*,XAXX010101000,0240119813000000,LEGISLATURA,NaN,130.0,186.0,744601.0,DE 01-2022 AL 12-2022,...,1086.0,120.0,NaN,120.0,NaN,NaN,2022,0240119800,0240119813,0240119813


In [145]:
def delete_empty(df):

    for i in df.columns:
        df.loc[(df[i] =="") |(df[i] =="0") | (df[i] ==0) | (df[i].isna()) | (df[i] ==" " ), i]=float('Nan')
    df.dropna(axis=1, how='all', inplace=True)
    
    return df 

In [155]:
delete_empty(base_predial)

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\3696090809.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, how='all', inplace=True)


,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,fec_pago,importe,recargos,multas,descuento,fol_fisc_uuid,facturado
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747.0,DE 01-2022 AL 12-2022,2022,2022-01-03,196.0,NaN,NaN,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
1,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,02401271010F0014,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,2022-01-03,219.0,NaN,NaN,18.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.0,58.0,621149.0,DE 01-2022 AL 12-2022,2022,2022-01-03,864.0,NaN,NaN,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.0,251.0,867817.0,DE 01-2022 AL 12-2022,2022,2022-01-03,1356.0,NaN,NaN,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.0,79.0,295397.0,DE 01-2022 AL 12-2022,2022,2022-01-03,384.0,NaN,NaN,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23131,TM049719,MONTOYA CRUZ MAURILIO,XAXX010101000,0240132737000000,"PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL","PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL",93.0,142.0,446012.0,DE 01-2022 AL 12-2022,2022,2022-12-30,594.0,87.0,NaN,NaN,NaN,NaN
23132,TM049720,BRENDA IVON MONROY OSORNIO,XAXX010101000,0240150021000000,"FELIPE VILLANUEVA, S/N, COL. EL JAZMIN","IGNACIO RAYON NO. 105, INT 7 COL. 5 DE MAYO, ...",157.0,NaN,112343.0,DE 01-2020 AL 12-2020,2020,2022-12-30,207.0,104.0,NaN,104.0,NaN,NaN
23135,TM049723,DIAZ RAMIREZ JESUS*,XAXX010101000,0240119813000000,LEGISLATURA,NaN,130.0,186.0,744601.0,DE 01-2022 AL 12-2022,2022,2022-12-30,1086.0,120.0,NaN,120.0,NaN,NaN
23136,TM049724,COLIN FLORES ELOISA*,XAXX010101000,0240107919000000,"ISIDRO FABELA SUR # 27 - S/N, COL. CENTRO, C.P...",NaN,250.0,79.0,382309.0,DE 01-2019 AL 12-2019,2019,2022-12-30,504.0,252.0,NaN,252.0,NaN,NaN


In [159]:
# base_predial.columns
base_predial.periodo_anual.value_counts()

2022    13894
2021      791
2017      475
2020      341
2019      158
2018      106
2016        1
Name: periodo_anual, dtype: int64

In [183]:
base_predial=base_predial[base_predial['EJERCICIO_FISCAL']== "2022"]
base_predial.shape

(13894, 19)

In [ ]:
base_predial=base_predial[base_predial['periodo_anual'] == '2022']
base_predial

In [ ]:
base_predial=base_predial[base_predial['EJERCICIO FISCAL'] == 2022]
base_predial

In [189]:
base_predial.columns[base_predial.columns.isin(manzanas_mun.columns)]
falta= base_predial.columns[~base_predial.columns.isin(manzanas_mun.columns)]
falta

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL'],
      dtype='object')

In [190]:
cruce_cat=pd.merge(base_predial[['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL']],CURT_mun, left_on="CLAVE_PREDIO", right_on="CLAVE_CATASTRAL", how='inner')
cruce_cat

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,...,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL_y,LATITUD,LONGITUD
0,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
1,IT765966,DURAN MARTINEZ ELVIRA,XAXX010101000,"MANUEL M. PONCE, S/N, COL. LAS FUENTES","MANUEL M. PONCE, S/N, COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
2,TM000028,ASOCIACION CIVIL DE MAESTROS X UNA VIVIENDA DI...,XAXX010101000,MANUEL M. PONCE 36 INT. 02 LAS FUENTES,NaN,172.0,156.0,741760.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
3,TM000582,MARTINEZ HERRERA ROSA*,XAXX010101000,MANUEL M. PONCE 36 INT 20,NaN,172.0,NaN,148780.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
4,TM005563,BRAULIO MARTINEZ GERMAN,XAXX010101000,"MANUEL M. PONCE # S/N - 30, COL. MIGUEL PORTIL...",NaN,172.0,48.0,319658.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,TM051879,GONZALEZ ROJAS ANDRES ALFONSO*,XAXX010101000,"LIC. MARIO COLIN SANCHEZ # S/N - S/N, COL. ATL...",NaN,1400.0,280.0,1757016.0,DE 01-2022 AL 12-2022,2022,...,None,024,01,091,0240109100000000,POINT (-99.88620 19.79345),0240109116000000,0240109116,19.793448,-99.886200
2347,TM051879,GONZALEZ ROJAS ANDRES ALFONSO*,XAXX010101000,"LIC. MARIO COLIN SANCHEZ # S/N - S/N, COL. ATL...",NaN,1400.0,280.0,1757016.0,DE 01-2022 AL 12-2022,2022,...,None,NaN,NaN,NaN,NaN,POINT (-99.88607 19.79354),0240109116000000,0240109116,19.793543,-99.886073
2348,TM049194,CARDENAS BECERRIL ARMANDO,XAXX010101000,"MERCADO GONZALEZ MA CONSUELO S/N, COL. LAS FUE...","MERCADO GONZALEZ MA CONSUELO S/N, COL. LAS FUE...",125.0,176.0,978002.0,DE 01-2022 AL 12-2022,2022,...,None,024,01,271,0240127100000000,POINT (-99.88202 19.80159),0240127141000000,0240127141,19.801593,-99.882019
2349,TM049325,NAVARRETE YAÑEZ ELISEO*,XAXX010101000,"A LAS ARENAS, TIERRAS BLANCAS",NaN,6856.0,NaN,185112.0,DE 01-2022 AL 12-2022,2022,...,None,024,08,056,0240805600000000,POINT (-99.87658 19.87062),0240805618000000,0240805618,19.870623,-99.876581


In [191]:
base_predial_n = base_predial.loc[~base_predial['CLAVE_PREDIO'].isin(cruce_cat['CLAVE_PREDIO'])]
base_predial_n

,fol_rec,nombre,rfc,CLAVECATASTRAL,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,...,importe,recargos,multas,descuento,fol_fisc_uuid,facturado,EJERCICIO_FISCAL,CLAVE_MANZ,CLAVE_PREDIO,CLAVE_CATASTRAL
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,0241101614000000,SANTIAGO ACUTZILAPAN,NaN,45.00,26.0,77747.0,DE 01-2022 AL 12-2022,...,196.0,NaN,NaN,16.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0241101600,0241101614,0241101614
2,IT758591,ROBERTO SUAREZ JULIA*,XAXX010101000,0240210222000000,SANTA MARIA NATIVITAS TECOAC,NaN,2988.00,58.0,621149.0,DE 01-2022 AL 12-2022,...,864.0,NaN,NaN,72.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240210200,0240210222,0240210222
3,IT755057,SANDOVAL FIGUEROA ANTONIO*,XAXX010101000,0240102207000000,ANTONIO MONROY YANEZ S/N,NaN,224.00,251.0,867817.0,DE 01-2022 AL 12-2022,...,1356.0,NaN,NaN,216.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240102200,0240102207,0240102207
4,IT758592,SUAREZ ESCOBAR JESUS,XAXX010101000,0240211027000000,"GUSTAVO BAZ PRADA, S/N, TECOAC","GUSTAVO BAZ PRADA, S/N, TECOAC",182.00,79.0,295397.0,DE 01-2022 AL 12-2022,...,384.0,NaN,NaN,31.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240211000,0240211027,0240211027
6,IT765965,A.CIVIL DE MAESTROS X 1 VIVIENDA D.,XAXX010101000,02401271160E0031,"PRIVADA ENRIQUE BATIZ, S/N, COL. LAS FUENTES","PRIVADA ENRIQUE BATIZ, S/N, COL. LAS FUENTES",171.99,64.0,389975.0,DE 01-2022 AL 12-2022,...,515.0,NaN,NaN,41.0,1769A2C0-F4B6-407E-9AFA-84D5F1652AD7,NaN,2022,0240127100,0240127116,0240127116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23129,TM049715,JIMENEZ CHAVEZ MARIA DEL REFUGIO*,XAXX010101000,0240121208000000,MORELOS JOSE MARIA PTE. COL. LA GARITA,NaN,239.00,309.0,1328705.0,DE 01-2022 AL 12-2022,...,2646.0,293.0,NaN,900.0,NaN,NaN,2022,0240121200,0240121208,0240121208
23130,TM049716,JIMENEZ DE MORALES MA. DEL REFUGIO*,XAXX010101000,0240152401000000,3 SANTA CRUZ BOMBATEVI,NaN,1463.00,80.0,866723.0,DE 01-2022 AL 12-2022,...,1356.0,151.0,NaN,NaN,NaN,NaN,2022,0240152400,0240152401,0240152401
23131,TM049719,MONTOYA CRUZ MAURILIO,XAXX010101000,0240132737000000,"PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL","PRIVADA DE LA MINA NO. 10, COL. DOS DE ABRIL",93.00,142.0,446012.0,DE 01-2022 AL 12-2022,...,594.0,87.0,NaN,NaN,NaN,NaN,2022,0240132700,0240132737,0240132737
23135,TM049723,DIAZ RAMIREZ JESUS*,XAXX010101000,0240119813000000,LEGISLATURA,NaN,130.00,186.0,744601.0,DE 01-2022 AL 12-2022,...,1086.0,120.0,NaN,120.0,NaN,NaN,2022,0240119800,0240119813,0240119813


In [192]:
base_predial_n.columns[base_predial_n.columns.isin(manzanas_mun.columns)]
falta= base_predial_n.columns[~base_predial_n.columns.isin(manzanas_mun.columns)]
falta

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL'],
      dtype='object')

In [193]:
cruce_manzana = pd.merge(base_predial_n[['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL']], CURT_mun, left_on='CLAVE_MANZ', right_on='CLAVE_CATASTRAL', how='inner')
cruce_manzana

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,...,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL_y,LATITUD,LONGITUD
0,IT755056,MEDINA GARCIA ARCELIA*,XAXX010101000,SANTIAGO ACUTZILAPAN,NaN,45.0,26.0,77747.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,11,016,0241101600000000,POINT (-99.76585 19.78757),0241101600000000,0241101600,19.787568,-99.765852
1,IT766308,RUBEN GARCIA PERUSQUIA,XAXX010101000,"AV. BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN","AV. BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN",166.0,NaN,74593.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,11,016,0241101600000000,POINT (-99.76585 19.78757),0241101600000000,0241101600,19.787568,-99.765852
2,TM005017,GUSTAVO ORDOÑEZ MEDINA,XAXX010101000,"BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN","BENITO JUAREZ, S/N, SANTIAGO ACUTZILAPAN",335.0,NaN,157450.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,11,016,0241101600000000,POINT (-99.76585 19.78757),0241101600000000,0241101600,19.787568,-99.765852
3,TM006685,ZALDIVAR GONZALEZ JOSE TRINIDAD *,XAXX010101000,CALLE JUAREZ SANTIAGO ACUTZILAPAN,NaN,100.0,40.0,145682.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,11,016,0241101600000000,POINT (-99.76585 19.78757),0241101600000000,0241101600,19.787568,-99.765852
4,TM002820,GARCIA GUADARRAMA LUCIANO*,XAXX010101000,JUAREZ SANTIAGO ACUTZILAPAN,NaN,284.0,200.0,568840.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,11,016,0241101600000000,POINT (-99.76585 19.78757),0241101600000000,0241101600,19.787568,-99.765852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12357,TM051613,ROSA YAÑEZ CARDENAS,YACR641216SY9,ISAIAS MONROY S/N,NaN,195.0,NaN,276900.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,0240116000,19.795031,-99.885823
12358,TM051616,ROSA YAÑEZ CARDENAS,YACR641216SY9,CALLE HACIENDA DE TOXI S/N FRACC. ATLAVILLA,"AV. ISIDRO FABELA NO. 15, ATLACOMULCO DE FABEL...",154.0,NaN,218680.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,0240116000,19.795031,-99.885823
12359,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,04,052,0240405200000000,POINT (-99.95662 19.79763),0240405200000000,0240405200,19.797632,-99.956616
12360,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.0,NaN,115943.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,07,212,0240721200000000,POINT (-99.87332 19.81364),0240721200000000,0240721200,19.813642,-99.873320


In [194]:
cruce_predial_manz = pd.concat([cruce_cat,cruce_manzana])
cruce_predial_manz

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,...,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL_y,LATITUD,LONGITUD
0,IT765964,ALFONSO SANCHEZ VERA,XAXX010101000,"MANUEL M. PONCE, S/N, COLONIA LAS FUENTES","MANUEL M. PONCE, S/N ,COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
1,IT765966,DURAN MARTINEZ ELVIRA,XAXX010101000,"MANUEL M. PONCE, S/N, COL. LAS FUENTES","MANUEL M. PONCE, S/N, COL. LAS FUENTES",172.0,NaN,148771.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
2,TM000028,ASOCIACION CIVIL DE MAESTROS X UNA VIVIENDA DI...,XAXX010101000,MANUEL M. PONCE 36 INT. 02 LAS FUENTES,NaN,172.0,156.0,741760.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
3,TM000582,MARTINEZ HERRERA ROSA*,XAXX010101000,MANUEL M. PONCE 36 INT 20,NaN,172.0,NaN,148780.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
4,TM005563,BRAULIO MARTINEZ GERMAN,XAXX010101000,"MANUEL M. PONCE # S/N - 30, COL. MIGUEL PORTIL...",NaN,172.0,48.0,319658.0,DE 01-2022 AL 12-2022,2022,...,14 _ Ver curtnotas,024,01,271,0240127100000000,POINT (-99.88399 19.80208),0240127101000000,0240127101,19.802084,-99.883991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12357,TM051613,ROSA YAÑEZ CARDENAS,YACR641216SY9,ISAIAS MONROY S/N,NaN,195.0,NaN,276900.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,0240116000,19.795031,-99.885823
12358,TM051616,ROSA YAÑEZ CARDENAS,YACR641216SY9,CALLE HACIENDA DE TOXI S/N FRACC. ATLAVILLA,"AV. ISIDRO FABELA NO. 15, ATLACOMULCO DE FABEL...",154.0,NaN,218680.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,0240116000,19.795031,-99.885823
12359,TM051701,GUTIERREZ MIRA EPITACIA CONCEPCION,XAXX010101000,"CALLE 5 DE MAYO, S/N, CUENDO",NaN,657.0,NaN,12155.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,04,052,0240405200000000,POINT (-99.95662 19.79763),0240405200000000,0240405200,19.797632,-99.956616
12360,TM049657,GARCIA VALVERDE PATRICIA,XAXX010101000,"CAMINO SANTA CRUZ, S/N, LA PALMA",NaN,579.0,NaN,115943.0,DE 01-2022 AL 12-2022,2022,...,NaN,024,07,212,0240721200000000,POINT (-99.87332 19.81364),0240721200000000,0240721200,19.813642,-99.873320


In [195]:
cruce_predial_manz.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL_x', 'cve_ent', 'cve_mun',
       'cve_loc', 'id_cat', 'curt', 'notas', 'mun', 'zona', 'manz', 'cve_cat',
       'geometry', 'CLAVECATASTRAL', 'CLAVE_CATASTRAL_y', 'LATITUD',
       'LONGITUD'],
      dtype='object')

In [ ]:
#Solo hay cruce a nivel manzana
cruce_predial_manz = cruce_manzana
cruce_predial_manz

In [196]:
no_cruzo_predialxmanz=base_predial.loc[~base_predial['CLAVE_CATASTRAL'].isin(cruce_predial_manz['CLAVE_CATASTRAL_x'])]
no_cruzo_predialxmanz.to_csv(r"D:\Secretaria\cruces_bases\Atlacomulco/Altacomulco_nocruzo_predialxmanz.csv", encoding="utf-8-sig")

In [197]:
cruce_predial_manz.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'CLAVE_CATASTRAL_x', 'cve_ent', 'cve_mun',
       'cve_loc', 'id_cat', 'curt', 'notas', 'mun', 'zona', 'manz', 'cve_cat',
       'geometry', 'CLAVECATASTRAL', 'CLAVE_CATASTRAL_y', 'LATITUD',
       'LONGITUD'],
      dtype='object')

In [198]:
cruce_predial_manz['CLAVE_CATASTRAL']=cruce_predial_manz.CLAVE_CATASTRAL_x.combine_first(cruce_predial_manz.CLAVE_CATASTRAL_y)
cruce_predial_manz[cruce_predial_manz.CLAVECATASTRAL.isna()]

,fol_rec,nombre,rfc,ubicacion,domicilio_fiscal,sup_terreno,sup_construc,valor_catastral,periodo,periodo_anual,...,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL_y,LATITUD,LONGITUD,CLAVE_CATASTRAL


In [199]:
#Eliminar columna de CLAVECATASTRAL_y y renombrar columna de CLAVECATASTRAL_x
cruce_predial_manz.drop(['CLAVE_CATASTRAL_y','CLAVE_CATASTRAL_x'],axis=1,inplace=True )
#cruce_cat.rename(columns={'CLAVECATASTRAL_x':'CLAVECATASTRAL'},inplace=True)
cruce_predial_manz.columns

Index(['fol_rec', 'nombre', 'rfc', 'ubicacion', 'domicilio_fiscal',
       'sup_terreno', 'sup_construc', 'valor_catastral', 'periodo',
       'periodo_anual', 'fec_pago', 'importe', 'recargos', 'multas',
       'descuento', 'fol_fisc_uuid', 'facturado', 'EJERCICIO_FISCAL',
       'CLAVE_MANZ', 'CLAVE_PREDIO', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat',
       'curt', 'notas', 'mun', 'zona', 'manz', 'cve_cat', 'geometry',
       'CLAVECATASTRAL', 'LATITUD', 'LONGITUD', 'CLAVE_CATASTRAL'],
      dtype='object')

In [200]:
#Guardar base de predialvscurt
cruce_predial_manz.to_csv(r"D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_cruce_predialxcurt.csv")

Lectura de las 3 bases de predial (Predial, propietario y deudores)

In [201]:
pred_pred=pd.read_excel(r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PREDIAL.xlsx")
pred_pred.tail(3)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,CLAVECATASTRAL,CLAVEASENTAMIENTO,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CLAVEMANZANA,CLAVEZONA,CLAVESTATUS,DIRECCION,VALORTERRENO,...,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,LATITUD,LONGITUD,FECHAALTA,ASENTAMIENTO_NR,CLAVECP_NR
29755,0241530823000000,555,50475,14,15,308,15,1,SIN NOMBRE S/N S/N,89910,...,810,0,0,0,0241530807000000,0,0,2016-03-04 15:23:03.0,,
29756,0241530824000000,554,50475,14,15,308,15,1,SIN NOMBRE S/N S/N,568004,...,9986,0,0,0,0241580410000000,0,0,2016-03-23 11:04:24.0,,
29757,0241530825000000,555,50475,14,15,308,15,1,Sin Nombre S/N,16095,...,145,0,0,0,,0,0,2017-05-03 18:06:18.0,,


In [202]:
pred_prop = pd.read_excel(r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PROPIETARIO.xlsx")
pred_prop.tail(3)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR
29755,MIRIAM,MELCHOR,DE LA CRUZ,,50475,14,15,SIN NOMBRE S/N S/N,0241530823000000,1,555,,,,
29756,FELIPE,HIPOLITO,RUIZ,,50475,14,15,SIN NOMBRE S/N S/N,0241530824000000,1,554,,,,
29757,ORDOÑEZ MENDOZA JUSTO,,,,50475,14,15,"CALLE SIN NOMBRE,# S/N,COL.SAN LUIS BORO,CP. 5...",0241530825000000,1,555,,,,


In [203]:
pred_deu = pd.read_excel(r"D:\Secretaria\cruces_bases\Atlacomulco\Atlacomulco_PREDIALADEUDO.xlsx")
pred_deu.tail(3)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
260989,0241530825000000,2018,1,12,16095,Adeudo
260990,0241530825000000,2019,1,12,17690,Adeudo
260991,0241530825000000,2021,1,12,21025,Adeudo


In [ ]:
def delete_empty(df):

    for i in df.columns:
        df.loc[(df[i] =="") |(df[i] =="0") | (df[i] ==0) | (df[i].isna()) | (df[i] ==" " ), i]=float('Nan')
    df.dropna(axis=1, how='all', inplace=True)
    
    return df 

In [204]:
#borrar columnas vacias
delete_empty(pred_prop)
delete_empty(pred_pred)
delete_empty(pred_deu)

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
0,0240000000PRUEBA,2018.0,1.0,12.0,245321.0,Adeudo
1,0240000000PRUEBA,2019.0,1.0,12.0,273408.0,Adeudo
2,0240000001PRUEBA,2012.0,1.0,12.0,22222.0,Adeudo
3,0240000001PRUEBA,2013.0,1.0,12.0,2222.0,Adeudo
4,0240000004PRUEBA,2011.0,1.0,12.0,400000.0,Adeudo
...,...,...,...,...,...,...
260987,0241530824000000,2019.0,1.0,12.0,624325.0,Pago Municipal
260988,0241530824000000,2021.0,1.0,12.0,742060.0,Adeudo
260989,0241530825000000,2018.0,1.0,12.0,16095.0,Adeudo
260990,0241530825000000,2019.0,1.0,12.0,17690.0,Adeudo


In [205]:
#borrar duplicados de las bases
pred_deu.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_pred.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)
pred_prop.drop_duplicates(subset='CLAVECATASTRAL', inplace=True)

In [206]:
pred_deu = pred_deu[pred_deu['EJERCICIO'] == 2021]
pred_deu

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
4319,0240103204000000,2021.0,1.0,12.0,837962.0,Pago Municipal
8813,0240106144000000,2021.0,1.0,12.0,52845.0,Adeudo
12202,0240107826000000,2021.0,1.0,12.0,304453.0,Pago Municipal
21735,0240111527000000,2021.0,1.0,12.0,232276.0,Pago Municipal
25410,0240113439000000,2021.0,1.0,12.0,1161317.0,Pago Municipal
...,...,...,...,...,...,...
253995,0241504162000000,2021.0,1.0,12.0,25955.0,Pago Municipal
254420,0241504411000000,2021.0,1.0,12.0,149930.0,Adeudo
257840,0241508207000000,2021.0,1.0,12.0,15660.0,Pago Municipal
258321,0241510017000000,2021.0,1.0,12.0,20300.0,Pago Municipal


In [207]:
pred_pred['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.astype(str).str[0:10]
pred_prop['CLAVE_CATASTRAL']= pred_prop.CLAVECATASTRAL.astype(str).str[0:10]
pred_deu['CLAVE_CATASTRAL']= pred_deu.CLAVECATASTRAL.astype(str).str[0:10]

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\2892822269.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_deu['CLAVE_CATASTRAL']= pred_deu.CLAVECATASTRAL.astype(str).str[0:10]


In [212]:
pred_pred.CLAVE_CATASTRAL

0        0240000000
1        0240000001
2        0240000004
3        0240100101
4        0240100201
            ...    
29753    0241530821
29754    0241530822
29755    0241530823
29756    0241530824
29757    0241530825
Name: CLAVE_CATASTRAL, Length: 29758, dtype: object

In [208]:
#Asegurarse que se cortaron todas las claves y no hay nan
pred_pred[pred_pred.CLAVE_CATASTRAL.isna()]
pred_prop[pred_prop.CLAVE_CATASTRAL.isna()]
pred_deu[pred_deu.CLAVE_CATASTRAL.isna()]

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL


Qué columnas están dentro de mi base x cruzar, que no están dentro de la 1ra base 

In [209]:
pred_pred.columns[pred_pred.columns.isin(pred_prop.columns)]
falta= pred_pred.columns[~pred_pred.columns.isin(pred_prop.columns)]
falta

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA'],
      dtype='object')

In [ ]:
pred_pred['CLAVECATASTRAL']=pred_pred.CLAVECATASTRAL.astype(str).str[0:16]
pred_pred.dtypes

Se hace el cruce con las columnas que no están dentro de la 1er base + la llave

In [213]:
#Cruce de predial completo con propietario (el resultado de este cruce es la primera parte de la base completa de predial, no debe tener duplicados por clave catastral unica a 16 digitos y esa es la key de cruce)
predial_general = pd.merge(pred_pred[['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA']], pred_prop, on="CLAVECATASTRAL", how="inner")
predial_general

,CLAVE_CATASTRAL_x,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL_y
0,0240000000,0240000000PRUEBA,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,...,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,0240000000
1,0240000001,0240000001PRUEBA,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,...,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,0240000001
2,0240000004,0240000004PRUEBA,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,...,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,0240000004
3,0240100101,0240100101000000,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,...,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,0240100101
4,0240100201,0240100201000000,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,...,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821,0241530821000000,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,...,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,0241530821
29754,0241530822,0241530822000000,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530822
29755,0241530823,0241530823000000,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530823
29756,0241530824,0241530824000000,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,0241530824


In [214]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
predial_general['CLAVE_CATASTRAL']=predial_general.CLAVE_CATASTRAL_x.combine_first(predial_general.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
predial_general = predial_general.drop(columns_to_drop, axis=1)
predial_general


,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL
0,0240000000PRUEBA,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,...,7.0,15.0,PRUEBA 1,1.0,1054.0,NaN,NaN,NaN,NaN,0240000000
1,0240000001PRUEBA,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,...,17.0,15.0,PRUEBA 2 2,1.0,9282.0,NaN,NaN,NaN,NaN,0240000001
2,0240000004PRUEBA,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,...,6.0,15.0,PRUEBA 111 111,1.0,6881.0,NaN,NaN,NaN,NaN,0240000004
3,0240100101000000,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,...,14.0,15.0,AV. ISIDRO FABELA SUR S/N,1.0,528.0,NaN,MAT,NaN,NaN,0240100101
4,0240100201000000,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,...,14.0,15.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENTRO,CP....",1.0,528.0,NaN,NaN,NaN,NaN,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821000000,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,...,14.0,15.0,SIN NOMBRE S/N,1.0,555.0,tathy311@gmail.com,SENM,NaN,NaN,0241530821
29754,0241530822000000,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530822
29755,0241530823000000,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,555.0,NaN,NaN,NaN,NaN,0241530823
29756,0241530824000000,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,...,14.0,15.0,SIN NOMBRE S/N S/N,1.0,554.0,NaN,NaN,NaN,NaN,0241530824


In [215]:
predial_general[predial_general.CLAVE_CATASTRAL.isna()]

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL


In [216]:
#revisión de columnas entre predial general (cruce 1) y predial adeudos
predial_general.columns[predial_general.columns.isin(pred_deu.columns)]
falta_deu= predial_general.columns[~predial_general.columns.isin(pred_deu.columns)]
falta_deu

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR'],
      dtype='object')

In [82]:
pred_deu['CLAVECATASTRAL']=pred_deu['CLAVECATASTRAL'].astype(str).str[0:16]
pred_deu.dtypes

C:\Users\mfpen\AppData\Local\Temp\ipykernel_16536\1043893139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_deu['CLAVECATASTRAL']=pred_deu['CLAVECATASTRAL'].astype(str).str[0:16]


CLAVECATASTRAL      object
EJERCICIO          float64
CLAVEPERIODOINI    float64
CLAVEPERIODOFIN    float64
VALORCATASTRAL     float64
STATUS              object
CLAVE_CATASTRAL     object
dtype: object

In [ ]:
predial_general.columns

In [217]:
cruce_pred1=pd.merge(predial_general[['CLAVECATASTRAL','CLAVE_CATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR']],pred_deu, on="CLAVECATASTRAL",  how='outer', indicator=True)
cruce_pred1

,CLAVECATASTRAL,CLAVE_CATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL_y,_merge
0,0240000000PRUEBA,0240000000,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,0240000001PRUEBA,0240000001,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0240000004PRUEBA,0240000004,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0240100101000000,0240100101,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,...,MAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0240100201000000,0240100201,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821000000,0241530821,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,...,SENM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29754,0241530822000000,0241530822,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29755,0241530823000000,0241530823,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29756,0241530824000000,0241530824,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [218]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_pred1['CLAVE_CATASTRAL']=cruce_pred1.CLAVE_CATASTRAL_x.combine_first(cruce_pred1.CLAVE_CATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVE_CATASTRAL_x', 'CLAVE_CATASTRAL_y']
cruce_pred1 = cruce_pred1.drop(columns_to_drop, axis=1)
cruce_pred1

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,_merge,CLAVE_CATASTRAL
0,0240000000PRUEBA,NaN,NaN,PRUEBA 1 1,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000000
1,0240000001PRUEBA,NaN,NaN,PRUEBA 2 2,NaN,NaN,NaN,NaN,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000001
2,0240000004PRUEBA,NaN,NaN,PRUEBA 111 111,300000,NaN,145000,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240000004
3,0240100101000000,1.0,1.0,ISIDRO FABELA SUR S/N,5245887,NaN,NaN,NaN,3813.0,NaN,...,MAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100101
4,0240100201000000,2.0,1.0,"AVENIDA FABELA ISIDRO NTE,# S/N,COL.CENT S/N S/N",4092285,NaN,3341520,NaN,5493.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0240100201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29753,0241530821000000,308.0,15.0,SIN NOMBRE S/N,41625,NaN,NaN,NaN,375.0,NaN,...,SENM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0241530821
29754,0241530822000000,308.0,15.0,SIN NOMBRE S/N S/N,12987,NaN,NaN,NaN,117.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0241530822
29755,0241530823000000,308.0,15.0,SIN NOMBRE S/N S/N,89910,NaN,NaN,NaN,810.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0241530823
29756,0241530824000000,308.0,15.0,SIN NOMBRE S/N S/N,568004,NaN,NaN,NaN,9986.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0241530824


In [219]:
cruce_pred1[cruce_pred1.CLAVE_CATASTRAL.isna()]

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,_merge,CLAVE_CATASTRAL


In [220]:
predial_completo = cruce_pred1[cruce_pred1['_merge'] =='both']
predial_sin_adeudos = cruce_pred1[cruce_pred1['_merge'] =='left_only']
predial_completo

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,_merge,CLAVE_CATASTRAL
519,0240103204000000,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,NaN,...,COBJ540720MMCLCC08,NaN,NaN,2021.0,1.0,12.0,837962.0,Pago Municipal,both,0240103204
1033,0240106144000000,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,GAGF701203MMCRLR02,NaN,NaN,2021.0,1.0,12.0,52845.0,Adeudo,both,0240106144
1427,0240107826000000,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,304453.0,Pago Municipal,both,0240107826
2572,0240111527000000,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,NaN,...,SALH570415MMCNPR02,NaN,NaN,2021.0,1.0,12.0,232276.0,Pago Municipal,both,0240111527
2999,0240113439000000,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,1161317.0,Pago Municipal,both,0240113439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28975,0241504162000000,41.0,15.0,A DIXIMOXI S/N,24165,NaN,NaN,NaN,179.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,25955.0,Pago Municipal,both,0241504162
29020,0241504411000000,44.0,15.0,CARRETERA A SAN IGNACIO Y LA JOYA S/N S/N,126148,NaN,NaN,NaN,1034.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,149930.0,Adeudo,both,0241504411
29412,0241508207000000,82.0,15.0,SIN NOMBRE S/N S/N,13176,NaN,NaN,NaN,108.0,NaN,...,SAMA780717HMCNNR01,NaN,NaN,2021.0,1.0,12.0,15660.0,Pago Municipal,both,0241508207
29471,0241510017000000,100.0,15.0,SIN NOMBRE S/N S/N,17080,NaN,NaN,NaN,140.0,NaN,...,MOPA890821HMCNXN05,NaN,NaN,2021.0,1.0,12.0,20300.0,Pago Municipal,both,0241510017


In [221]:
#guardar bases
predial_completo.to_csv(r'D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_predial_completo.csv',encoding="utf-8-sig")
predial_sin_adeudos.to_csv(r'D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_predial_completo_noadeudos.csv',encoding="utf-8-sig")

In [222]:
#eliminar la columna de merge en la base a trabajar de predial completo
predial_completo=predial_completo.drop(['_merge'], axis=1)

In [223]:
predial_completo.columns[predial_completo.columns.isin(cruce_predial_manz.columns)]
falta_pred_completo= predial_completo.columns[~predial_completo.columns.isin(cruce_predial_manz.columns)]
falta_pred_completo

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS'],
      dtype='object')

In [224]:
cruce_CURT_predial = pd.merge(predial_completo[['CLAVE_CATASTRAL', 'CLAVECATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS']], cruce_predial_manz, on='CLAVE_CATASTRAL', how = 'outer', indicator=True)
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVECATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,notas,mun,zona,manz,cve_cat,geometry,CLAVECATASTRAL_y,LATITUD,LONGITUD,_merge
0,0240103204,0240103204000000,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,...,NaN,024,01,032,0240103200000000,POINT (-99.86994 19.79901),0240103200000000,19.799013,-99.869940,both
1,0240106144,0240106144000000,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
2,0240107826,0240107826000000,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,...,NaN,024,01,078,0240107800000000,POINT (-99.87667 19.79462),0240107800000000,19.794622,-99.876672,both
3,0240111527,0240111527000000,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,...,NaN,024,01,115,0240111500000000,POINT (-99.87571 19.79685),0240111500000000,19.796849,-99.875711,both
4,0240113439,0240113439000000,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,...,None,024,01,134,0240113400000000,POINT (-99.87392 19.80178),0240113439000000,19.801782,-99.873920,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,0240116002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,19.795031,-99.885823,right_only
14757,0240116004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),0240116000000000,19.795031,-99.885823,right_only
14758,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,04,052,0240405200000000,POINT (-99.95662 19.79763),0240405200000000,19.797632,-99.956616,right_only
14759,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,07,212,0240721200000000,POINT (-99.87332 19.81364),0240721200000000,19.813642,-99.873320,right_only


In [225]:
#Al cruzar se generara x y y al existir la columna simultaneamente en las bases, se dejará la que tenga mayor peso
cruce_CURT_predial['CLAVECATASTRAL']=cruce_CURT_predial.CLAVECATASTRAL_x.combine_first(cruce_CURT_predial.CLAVECATASTRAL_y)
#ya que se tiene la columna correcta, se eliminan las generadas como x y Y
columns_to_drop = ['CLAVECATASTRAL_x', 'CLAVECATASTRAL_y']
cruce_CURT_predial = cruce_CURT_predial.drop(columns_to_drop, axis=1)
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,notas,mun,zona,manz,cve_cat,geometry,LATITUD,LONGITUD,_merge,CLAVECATASTRAL
0,0240103204,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,NaN,...,NaN,024,01,032,0240103200000000,POINT (-99.86994 19.79901),19.799013,-99.869940,both,0240103204000000
1,0240106144,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240106144000000
2,0240107826,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,NaN,...,NaN,024,01,078,0240107800000000,POINT (-99.87667 19.79462),19.794622,-99.876672,both,0240107826000000
3,0240111527,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,NaN,...,NaN,024,01,115,0240111500000000,POINT (-99.87571 19.79685),19.796849,-99.875711,both,0240111527000000
4,0240113439,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,NaN,...,None,024,01,134,0240113400000000,POINT (-99.87392 19.80178),19.801782,-99.873920,both,0240113439000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,0240116002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),19.795031,-99.885823,right_only,0240116000000000
14757,0240116004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,POINT (-99.88582 19.79503),19.795031,-99.885823,right_only,0240116000000000
14758,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,04,052,0240405200000000,POINT (-99.95662 19.79763),19.797632,-99.956616,right_only,0240405200000000
14759,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,07,212,0240721200000000,POINT (-99.87332 19.81364),19.813642,-99.873320,right_only,0240721200000000


In [229]:
cruce_CURT_predial= cruce_CURT_predial.rename(columns = {'_merge' : 'ID_predial'})

In [230]:
#Revisar si la base tiene LAT y LON
cruce_CURT_predial[cruce_CURT_predial.LONGITUD.isna()]


,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,notas,mun,zona,manz,cve_cat,geometry,LATITUD,LONGITUD,ID_predial,CLAVECATASTRAL
1,0240106144,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240106144000000
6,0240118726,187.0,1.0,CALLE ING. LUIS GALINDO RUIZ S/N S/N,61335,NaN,382200,NaN,87.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240118726000000
7,0240118727,187.0,1.0,ING. SAMUEL LOPEZ SIERRA S/N S/N,33347,NaN,95550,NaN,43.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240118727000000
8,0240119175,191.0,1.0,ADOLFO LOPEZ MATEOS S/N,180225,NaN,NaN,NaN,267.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240119175000000
12,0240119849,198.0,1.0,BENITO JUAREZ S/N,236925,NaN,NaN,NaN,351.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240119849000000
16,0240122371,223.0,1.0,CARRETERA A TOLUCA S/N S/N,214000,NaN,NaN,NaN,200.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240122371000000
33,0240140256,402.0,1.0,AVENIDA ZACUALPAN S/N S/N,1138322,NaN,NaN,NaN,12641.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240140256000000
34,0240140262,402.0,1.0,CARRETERA A JILOTEPEC S/N S/N,655000,NaN,NaN,NaN,655.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240140262000000
36,0240144240,442.0,1.0,AVENIDA DE LOS INSURGENTES S/N S/N,278270,NaN,NaN,NaN,480.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240144240000000
37,0240145228,452.0,1.0,LA MORA S/N S/N,85850,NaN,NaN,NaN,101.0,NaN,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,left_only,0240145228000000


Agregar LAT LON a los registros que no venían en la base de CURT

In [231]:
cruce_CURT_predial.loc[cruce_CURT_predial['LATITUD'].isna(), 'LATITUD'] = 18
cruce_CURT_predial.loc[cruce_CURT_predial['LONGITUD'].isna(), 'LONGITUD'] = -99

In [232]:
cruce_CURT_predial.drop("geometry",axis=1,inplace=True)

In [233]:
cruce_CURT_predial = prep.transform_df_to_gpd(cruce_CURT_predial, 'LONGITUD','LATITUD', crs= 'EPSG:4326')
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,notas,mun,zona,manz,cve_cat,LATITUD,LONGITUD,ID_predial,CLAVECATASTRAL,geometry
0,0240103204,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,NaN,...,NaN,024,01,032,0240103200000000,19.799013,-99.869940,both,0240103204000000,POINT (-99.86994 19.79901)
1,0240106144,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,left_only,0240106144000000,POINT (-99.00000 18.00000)
2,0240107826,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,NaN,...,NaN,024,01,078,0240107800000000,19.794622,-99.876672,both,0240107826000000,POINT (-99.87667 19.79462)
3,0240111527,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,NaN,...,NaN,024,01,115,0240111500000000,19.796849,-99.875711,both,0240111527000000,POINT (-99.87571 19.79685)
4,0240113439,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,NaN,...,None,024,01,134,0240113400000000,19.801782,-99.873920,both,0240113439000000,POINT (-99.87392 19.80178)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,0240116002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503)
14757,0240116004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503)
14758,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,04,052,0240405200000000,19.797632,-99.956616,right_only,0240405200000000,POINT (-99.95662 19.79763)
14759,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,024,07,212,0240721200000000,19.813642,-99.873320,right_only,0240721200000000,POINT (-99.87332 19.81364)


In [234]:
cruce_CURT_predial[cruce_CURT_predial.geometry.isna()]

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,notas,mun,zona,manz,cve_cat,LATITUD,LONGITUD,ID_predial,CLAVECATASTRAL,geometry


In [235]:
#exportar base 
# cruce_CURT_predial.to_excel(r"/Users/alfredoemiliomartinezcarbajal/Downloads/Atlacomulco_PREDIAL/cruce_curt_predial_Atlacomulco.xlsx")
cruce_CURT_predial.to_csv(r"D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_cruce_curtvspredial.csv",encoding='utf-8-sig',index=False)


In [ ]:
cruce_CURT_predial = pd.read_csv(r"D:\Secretaría\cruces_bases\Ixtapan/cruce_curt_predial_Ixtapan.csv",encoding='utf-8-sig')
cruce_CURT_predial

In [236]:
base_temp= CURT_mun.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

,CLAVE_CATASTRAL,CLAVECATASTRAL,curt
0,0240100100,1,0
1,0240100200,1,0
2,0240100300,1,0
3,0240100400,1,0
4,0240100500,1,0
...,...,...,...
3414,0241530817,1,1
3415,0241530822,1,1
3416,0241530824,1,1
3417,0241532000,1,0


In [237]:
# cruce_CURT_predial['ESTIMADO'] = cruce_CURT_predial['CLAVE_CATASTRAL'].value_counts().reset_index()['CLAVE_CATASTRAL']
base_temp=cruce_CURT_predial.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

,CLAVE_CATASTRAL,CLAVECATASTRAL,curt
0,0240100202,1,0
1,0240100301,27,0
2,0240100501,1,0
3,0240100502,1,0
4,0240100503,1,0
...,...,...,...
11202,0241532018,1,0
11203,0241532021,1,0
11204,0241532022,1,0
11205,0241532024,1,0


In [238]:
base_temp=base_temp.rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
base_temp

,CLAVE_CATASTRAL,ESTIMADO,curt
0,0240100202,1,0
1,0240100301,27,0
2,0240100501,1,0
3,0240100502,1,0
4,0240100503,1,0
...,...,...,...
11202,0241532018,1,0
11203,0241532021,1,0
11204,0241532022,1,0
11205,0241532024,1,0


In [239]:
crucexcurt=cruce_CURT_predial.merge(base_temp[['ESTIMADO','curt','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,zona,manz,cve_cat,LATITUD,LONGITUD,ID_predial,CLAVECATASTRAL,geometry,ESTIMADO,curt_y
0,0240103204,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,NaN,...,01,032,0240103200000000,19.799013,-99.869940,both,0240103204000000,POINT (-99.86994 19.79901),1,0
1,0240106144,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,NaN,NaN,NaN,18.000000,-99.000000,left_only,0240106144000000,POINT (-99.00000 18.00000),1,0
2,0240107826,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,NaN,...,01,078,0240107800000000,19.794622,-99.876672,both,0240107826000000,POINT (-99.87667 19.79462),1,0
3,0240111527,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,NaN,...,01,115,0240111500000000,19.796849,-99.875711,both,0240111527000000,POINT (-99.87571 19.79685),1,0
4,0240113439,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,NaN,...,01,134,0240113400000000,19.801782,-99.873920,both,0240113439000000,POINT (-99.87392 19.80178),1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,0240116002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503),1,0
14757,0240116004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503),1,0
14758,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,04,052,0240405200000000,19.797632,-99.956616,right_only,0240405200000000,POINT (-99.95662 19.79763),1,0
14759,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,07,212,0240721200000000,19.813642,-99.873320,right_only,0240721200000000,POINT (-99.87332 19.81364),1,0


In [240]:
# crucexcurt.drop("curt_y",axis=1,inplace=True)
crucexcurt.rename(columns={'curt_y':'CURT', "curt_x":"curt"},inplace=True)
crucexcurt

,CLAVE_CATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,zona,manz,cve_cat,LATITUD,LONGITUD,ID_predial,CLAVECATASTRAL,geometry,ESTIMADO,CURT
0,0240103204,32.0,1.0,CALLE 5 DE FEBRERO S/N S/N,726643,NaN,NaN,NaN,636.0,NaN,...,01,032,0240103200000000,19.799013,-99.869940,both,0240103204000000,POINT (-99.86994 19.79901),1,0
1,0240106144,61.0,1.0,20 DE NOVIEMBRE S/N S/N,45825,NaN,NaN,NaN,39.0,NaN,...,NaN,NaN,NaN,18.000000,-99.000000,left_only,0240106144000000,POINT (-99.00000 18.00000),1,0
2,0240107826,78.0,1.0,CALLE DE LA ASCENCION S/N S/N,161320,NaN,114637,NaN,218.0,NaN,...,01,078,0240107800000000,19.794622,-99.876672,both,0240107826000000,POINT (-99.87667 19.79462),1,0
3,0240111527,115.0,1.0,CALLE EMILIANO ZAPATA PTE S/N S/N,98700,NaN,114310,NaN,84.0,NaN,...,01,115,0240111500000000,19.796849,-99.875711,both,0240111527000000,POINT (-99.87571 19.79685),1,0
4,0240113439,134.0,1.0,ISIDRO FABELA NTE S/N,982386,NaN,118392,NaN,855.0,NaN,...,01,134,0240113400000000,19.801782,-99.873920,both,0240113439000000,POINT (-99.87392 19.80178),1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14756,0240116002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503),1,0
14757,0240116004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,01,160,0240116000000000,19.795031,-99.885823,right_only,0240116000000000,POINT (-99.88582 19.79503),1,0
14758,0240405203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,04,052,0240405200000000,19.797632,-99.956616,right_only,0240405200000000,POINT (-99.95662 19.79763),1,0
14759,0240721201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,07,212,0240721200000000,19.813642,-99.873320,right_only,0240721200000000,POINT (-99.87332 19.81364),1,0


In [241]:
crucexcurt.to_csv(r'D:\Secretaria\cruces_bases\Atlacomulco/Atlacomulco_Base_predial_PREGEO.csv',encoding="utf-8-sig", index=False)

#Prep data

In [ ]:
geom_novacios=crucexcurt[crucexcurt.LATITUD != 18 ]

In [ ]:
geom_novacios.to_csv(r'D:\Secretaría\cruces_bases\Ixtapan/geometry_no_vacia.csv',encoding="utf-8-sig", index=False)

Volver a leer CURT_mun

In [ ]:
#base de curt y manzanas municipales
CURT_mun=gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\CURT\060 Ixtapan de la Sal\predios_060.shp")
manzanas_mun = gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\Full manzanas\Full manzanas\Manzana_Ixtapan_Sal.shp")
#reemplazar id cat por clave catastral
CURT_mun['CLAVECATASTRAL']= CURT_mun['id_cat']
manzanas_mun['CLAVECATASTRAL']= manzanas_mun['cve_cat']

In [ ]:
#Disolución de las geometrías a partir de clavecatastral 
CURT_mun=CURT_mun.dissolve(by='CLAVECATASTRAL').reset_index()
CURT_mun

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT_mun.crs)
print("crs Manzanas: ",manzanas_mun.crs)

In [ ]:
#Se le asigna el crs de CURT a la base de manzanas
manzanas_mun=manzanas_mun.to_crs(4326)
# Se hace un overlay para la unión de las bases 
CURT_mun=CURT_mun.overlay(manzanas_mun, how='union',keep_geom_type=False)
CURT_mun

In [ ]:
CURT_mun['CLAVECATASTRAL_1']=CURT_mun.CLAVECATASTRAL_1.combine_first(CURT_mun.CLAVECATASTRAL_2)
CURT_mun

In [ ]:
def combine_first_drop(gdf):
    for i in (gdf.columns):
        if str('_1') in i :
            n = i
        if str('_2') in i :
            k = i
            z = i[:i.rfind('_1')-1]
            gdf[z] = gdf[n]
            gdf.drop(columns=[n,k],inplace=True)
        else:
            pass

In [ ]:
#Se utiliza esta función para renombrar columna con _1 y eliminar la otra
combine_first_drop(CURT_mun)
CURT_mun

In [ ]:
def separar_digitos(prueba):

    prueba.CLAVECATASTRAL = prueba.CLAVECATASTRAL.astype(str)
    for i in prueba.CLAVECATASTRAL:
        if str('000000') in i:
            cat = i
            cat=cat[:-6]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat
        elif str('00000000') in i :
            cat2=i
            cat2=cat2[:-8]
            prueba.loc[prueba.CLAVECATASTRAL== i, 'CLAVE_CATASTRAL']=cat2
        else:
            pass


In [ ]:
#Se utiliza esta función para separar CC en CC manzana y CC predio
separar_digitos(CURT_mun)
CURT_mun

In [ ]:
CURT_mun = CURT_mun.set_geometry('geometry')
CURT_mun

In [ ]:
test_igecem_final=CURT_mun[['geometry']].sjoin(geom_novacios)
test_igecem_final

In [ ]:
test_igecem_final.drop('cruce_predial_2_CURT',axis=1,inplace=True)

In [ ]:
test_igecem_final.columns

In [ ]:
#En el caso de que la base tenga alguna columna con acentos, se tiene que cambiar o no se podrá guardar como shp
test_igecem_final.rename(columns={'USO DE ÁREA HOMOGÉNEA':'USO DE AREA HOMOGENEA'},inplace=True)

In [ ]:
#Es para quitar index y columnas duplicadas x mismo nombre
test_igecem_final.drop(columns= test_igecem_final.columns[(test_igecem_final.columns.str.contains('index'))|(test_igecem_final.columns.duplicated())], inplace=True)

In [ ]:
test_igecem_final.drop(columns=['FECHAALTA','EJERCICIO FISCAL'], inplace=True)

In [ ]:
test_igecem_final=test_igecem_final.set_geometry('geometry')

In [ ]:
test_igecem_final.crs=4326

In [ ]:
test_igecem_final.geometry

In [ ]:
test_igecem_final.to_file(r"D:\Secretaría\cruces_bases\Ixtapan/test_igecem_final_Ixtapan.shp")
#cols_buenas.columns

In [ ]:
from shapely.geometry import MultiPolygon
test_igecem_final['detect_mpol']=test_igecem_final["geometry"].map(lambda y: isinstance(y,MultiPolygon))
test_igecem_final.loc[(test_igecem_final.detect_mpol == False) & (test_igecem_final.detect_pol == False )]

In [ ]:
linestring=test_igecem_final.loc[(test_igecem_final.detect_mpol == False) & (test_igecem_final.detect_pol == False )].dissolve()

In [ ]:
test_igecem=test_igecem_final[test_igecem_final.cve_cat != '0600120100000000']

In [ ]:
test_igecem = pd.concat([test_igecem,linestring])

In [ ]:
test_igecem.to_file(r"D:\Secretaría\cruces_bases\Ixtapan/test_igecem_final_Ixtapan.shp")

In [ ]:
cols_buenas=test_igecem_final.columns
cols_buenas

In [ ]:
cols_malas = gpd.read_file(r"D:\Secretaría\cruces_bases\Ixtapan/test_igecem_final_Ixtapan.shp")


In [ ]:
cols_malas.columns

Pasos para cruzar x cvegeo

In [ ]:
#Leer shape INEGI
inegi = gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\zonas_rurales_urbanas\15m.shp")
inegi

In [ ]:
lerma= inegi[inegi.CVE_MUN == '051']

In [ ]:
crucexzona=gpd.sjoin(lerma,CURT_mun)

In [ ]:
crucexzona.columns

In [ ]:
base_temp1= CURT_mun.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp1

In [ ]:
base_temp1=base_temp1.rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
base_temp1

In [ ]:
crucexcurt=CURT_mun.merge(base_temp1[['ESTIMADO','curt','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

In [ ]:

# crucexcurt.drop("curt_y",axis=1,inplace=True)
crucexcurt.rename(columns={'curt_y':'CURT', "curt_x":"curt"},inplace=True)
crucexcurt


In [ ]:
crucexcurt.to_csv(r'D:\Secretaría\cruces_bases\Lerma/Base_predial_PREGEO_Lerma.csv',encoding="utf-8-sig", index=False)